# Prepare environment

## Configure jupyter

* Install libs

In [ ]:
# %%capture
%pip install python-dotenv
%pip install psycopg2-binary

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Setting properties
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")
os.environ["PINECONE_ENV"] = os.getenv("PINECONE_ENV")

## Init Transformers

In [ ]:
# %%capture
%pip install transformers sentence-transformers

## Init Llama index

* https://docs.llamaindex.ai/en/stable/understanding/using_llms/using_llms/

In [ ]:
# %%capture
%pip install llama-index

### LLM Choices

#### For Ollama

* https://docs.llamaindex.ai/en/stable/examples/embeddings/ollama_embedding/

##### Ollama startup & verify

* https://replicate.com/blog/run-llama-locally

Start & check Ollama using

* http://localhost:11434/v1/chat/completions

```
{
  "model": "mistral:instruct",
  "messages": [
    {
      "role": "system",
      "content": "You are a helpful assistant."
    },
    {
      "role": "user",
      "content": "Hello!"
    }
  ]
}
```

In [ ]:
# %%capture
%pip install ollama
%pip install llama-index-llms-ollama
%pip install llama-index-embeddings-ollama

In [ ]:
# Starting from here, run ollama
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

llm_model = "mistral"
llm_url = "http://localhost:11434"

# Embedding Model to do local embedding using Ollama.
embedding = OllamaEmbedding(
    model_name=llm_model,
    base_url=llm_url,
    ollama_additional_kwargs={"mirostat": 0},
)
response = embedding.get_text_embedding_batch(
    ["This is a passage!", "This is another passage"], show_progress=True
)
# print(response)
query_embedding = embedding.get_query_embedding("Where is blue?")
# print(query_embedding)

# Completion
llm = Ollama(model=llm_model, base_url=llm_url, request_timeout=60.0)
response = llm.complete("Who is Laurie Voss? write in 10 words")
print(response)

#### LLM - For OpenAI

* https://docs.llamaindex.ai/en/stable/examples/llm/openai/
* https://llamahub.ai/l/llms/llama-index-llms-openai

In [ ]:
# %%capture

# https://platform.openai.com/docs/libraries
%pip install --upgrade openai

In [ ]:
import openai  # for calling the OpenAI API

# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.8)

### Vector stores

#### For ElasticSearch

* https://docs.llamaindex.ai/en/stable/examples/vector_stores/ElasticsearchIndexDemo/

In [ ]:
# %%capture
%pip install llama-index-vector-stores-elasticsearch elasticsearch

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.elasticsearch import ElasticsearchStore

# VectorStore that takes care of ES Index and Data management.
vector_store = ElasticsearchStore(index_name="conversation",
                 es_url="http://localhost:9200",
                 # es_cloud_id=os.getenv("ELASTIC_CLOUD_ID"),
                 # es_api_key=os.getenv("ELASTIC_API_KEY")
               )


### Rerank

#### For Cohere

In [ ]:
# %%capture
%pip install cohere llama-index-postprocessor-cohere-rerank

In [ ]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

## Load documents

In [ ]:
!mkdir -p '/data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O '/data/paul_graham/paul_graham_essay.txt'

In [ ]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()
documents

In [ ]:
# initialize without metadata filter
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)